# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,4.73,90,34,2.90,CL,1712700660
1,1,isafjordur,66.0755,-23.1240,-2.23,91,100,2.77,IS,1712700877
2,2,okha,53.5739,142.9478,-8.19,91,22,4.47,RU,1712700877
3,3,as sulayyil,20.4607,45.5779,27.14,49,88,7.54,SA,1712700878
4,4,mount gambier,-37.8333,140.7667,13.84,85,93,2.13,AU,1712700878


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
# %%capture --no-display

# Configure the map plot
mat_plot_1 = city_data_df.hvplot.points("Lng","Lat", geo = True, tiles = "OSM", frame_width = 1000, frame_height = 800, size= "Humidity", scale= 1.00, color= "City")

# Display the map
mat_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values. My ideal vacation will be based on high humidity (>=80) and 50% of less cloudiness. Temparatures between 19 and 32 degress
y = city_data_df.loc[(city_data_df["Humidity"]>= 30) & (city_data_df["Cloudiness"]< 50) & (city_data_df["Max Temp"]<32) & (city_data_df["Max Temp"]> 19)]
# https://saturncloud.io/blog/how-to-use-pandas-loc-with-multiple-conditions/

#Check if my database has Null values
null_columns = y.isnull().any(axis=1)
null_rows = y[null_columns]

# Drop any rows with null values
potential_cities_df =y.dropna()

# Display sample data
potential_cities_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,jamestown,42.0970,-79.2353,22.80,40,20,5.14,US,1712700457
6,6,laguna,38.4210,-121.4238,25.42,39,0,2.06,US,1712700878
25,25,bethel,41.3712,-73.4140,24.05,33,0,1.34,US,1712700884
34,34,adamstown,-25.0660,-130.1015,23.65,79,43,2.08,PN,1712700887
41,41,wailua homesteads,22.0669,-159.3780,26.24,78,40,7.20,US,1712700889
...,...,...,...,...,...,...,...,...,...,...
496,496,iskushuban,10.2837,50.2300,24.38,80,47,3.48,SO,1712701068
513,513,malvan,16.0667,73.4667,25.68,73,2,3.24,IN,1712701073
516,516,lautoka,-17.6167,177.4667,29.92,74,40,2.57,FJ,1712701014
530,530,tambura,5.6000,27.4667,24.53,54,26,1.82,SS,1712701078


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
#https://stackoverflow.com/questions/34682828/extracting-specific-selected-columns-to-new-dataframe-as-a-copy --> link used to find panda copy functions
hotel_df = potential_cities_df[["City","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
#https://www.delftstack.com/es/howto/python-pandas/how-to-create-an-empty-column-in-pandas-dataframe/ --> How to create an aditional column with empty values
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Hotel Name
5,jamestown,42.0970,-79.2353,40,
6,laguna,38.4210,-121.4238,39,
25,bethel,41.3712,-73.4140,33,
34,adamstown,-25.0660,-130.1015,79,
41,wailua homesteads,22.0669,-159.3780,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
# Add parameters as per API documentation
# https://apidocs.geoapify.com/playground/places/?map=12%2F48.4615%2F-89.2021&categories=%5B%22accommodation.hotel%22%5D&conditions=%5B%5D
params = {"categories":categories,
        "limit":20,
        "apiKey":geoapify_key,
} 
          

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]

    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
jamestown - nearest hotel: DoubleTree Jamestown
laguna - nearest hotel: Holiday Inn Express & Suites
bethel - nearest hotel: Hampton Inn Danbury
adamstown - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
callao - nearest hotel: Casa Ronald
beroroha - nearest hotel: No hotel found
luang prabang - nearest hotel: Amantaka
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
isangel - nearest hotel: Tanna Lodge
alofi - nearest hotel: Matavai Resort
lihue - nearest hotel: Kauai Palms
cockburn town - nearest hotel: The Salt Raker Inn
lompoc - nearest hotel: Red Roof Inn Lompoc
nova sintra - nearest hotel: Residência Ka Dencho
joinville - nearest hotel: Blue Tree Towers Joinville
namibe - nearest hotel: Nelsal Pensao
pisco - nearest hotel: La Portada
piton saint-leu - nearest hotel: Appartement Piton Saint Leu
cabo san lucas - nearest hotel: Comfort Rooms
ondjiva - nearest hotel: Combuesa
flying fish cove

,City,Lat,Lng,Humidity,Hotel Name
5,jamestown,42.0970,-79.2353,40,DoubleTree Jamestown
6,laguna,38.4210,-121.4238,39,Holiday Inn Express & Suites
25,bethel,41.3712,-73.4140,33,Hampton Inn Danbury
34,adamstown,-25.0660,-130.1015,79,No hotel found
41,wailua homesteads,22.0669,-159.3780,78,Hilton Garden Inn Kauai Wailua Bay
...,...,...,...,...,...
496,iskushuban,10.2837,50.2300,80,No hotel found
513,malvan,16.0667,73.4667,73,Hotel Silver Sand
516,lautoka,-17.6167,177.4667,74,Cathay Hotel
530,tambura,5.6000,27.4667,54,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot, adding the Hotel Name and the City.
#https://stackoverflow.com/questions/59678780/show-extra-columns-when-hovering-in-a-scatter-plot-with-hvplot --> Used to find out how to add columns in the chart, such as Hotel Name
mat_plot_2 = hotel_df.hvplot.points("Lng","Lat", geo = True, tiles = "OSM", frame_width = 1000, frame_height = 800, size= "Humidity", scale= 2.00, color= "City", hover_cols=["Country","Hotel Name"])

# Display the map
mat_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)